In [ ]:
import pytesseract, cv2
from ultralytics import YOLO
from pathlib import Path

# Point pytesseract at the engine you installed:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Load your trained detector
model = YOLO(r"runs/detect/train2/weights/best.pt")


In [ ]:
def filter_boxes_with_ocr(result, min_chars=3):
    """
    Return list of YOLO Boxes that contain >= min_chars alphanumerics
    according to Tesseract.
    """
    kept = []
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        # crop from original BGR image
        crop = result.orig_img[y1:y2, x1:x2]

        # Tesseract expects RGB or grayscale
        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

        text = pytesseract.image_to_string(crop_rgb, config="--psm 7")
        # Keep if enough alphanumerics
        if sum(ch.isalnum() for ch in text) >= min_chars:
            kept.append(box)
    return kept


In [ ]:
# Pick one test image
img_path = r"F:/license plate/images/val/video_15.jpg"

# Get prediction
result = model(img_path)[0]              # result for that one image

# Filter detections
good_boxes = filter_boxes_with_ocr(result, min_chars=3)

print(f"YOLO found {len(result.boxes)} boxes, OCR kept {len(good_boxes)}")

# Draw only the kept boxes
plotted = result.plot(boxes=good_boxes)  # plot() accepts an iterable of Boxes

cv2.imwrite(r"F:/license_plate_results/ocr_filtered.jpg", plotted)
print("✅ Saved annotated image with OCR‑filtered boxes.")


In [ ]:
img_dir = Path(r"F:/license plate/images/val")
out_dir = Path(r"F:/license_plate_results/ocr_filtered_imgs")
out_dir.mkdir(parents=True, exist_ok=True)

for p in img_dir.glob("*.jpg"):
    r = model(str(p))[0]
    good = filter_boxes_with_ocr(r, min_chars=3)
    annotated = r.plot(boxes=good)
    cv2.imwrite(str(out_dir / p.name), annotated)

print("✅ Done — OCR‑filtered images written to", out_dir)
